In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

rm: cannot remove '*.tsv.gz': No such file or directory
total 1155920
drwxr-xr-x 1 root root      4096 Jul 30 07:33 .
drwxr-xr-x 1 root root      4096 Jul 30 07:10 ..
drwxr-xr-x 4 root root      4096 Jul 27 13:36 .config
-rw-r--r-- 1 root root 251516986 Jul 29 13:25 name.basics.tsv.gz
drwxr-xr-x 1 root root      4096 Jul 27 13:37 sample_data
-rw-r--r-- 1 root root 310734144 Jul 29 13:25 title.akas.tsv.gz
-rw-r--r-- 1 root root 175266873 Jul 28 13:16 title.basics.tsv.gz
-rw-r--r-- 1 root root 446110311 Jul 29 13:25 title.principals.tsv.gz


In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
!pip install scikit-network==0.24.0

  Using cached scikit-network-0.24.0.tar.gz (1.7 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
title=pd.read_csv('title.basics.tsv.gz',sep='\t',low_memory=False).set_index('tconst')[['titleType','primaryTitle','startYear']]
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [ ]:
title.dtypes

titleType       object
primaryTitle    object
startYear       object
dtype: object

In [ ]:
title = title[title['startYear'] > '1999'] # filter by year

In [ ]:
import pandas as pd

# Set the chunk size (number of rows to read per batch)
chunk_size = 10000

# Function to process each batch of data
def process_batch(batch):
    # Filter the rows for the 'actor' category
    batch = batch[batch['category'] == 'actor']
    return batch

# File path for the data (replace with your actual file path)
file_path = 'title.principals.tsv.gz'

# Create an empty list to store the processed data chunks
processed_chunks = []

# Read the data in batches and process each batch
for batch in pd.read_csv(file_path, sep="\t", usecols=['tconst', 'nconst', 'category'], chunksize=chunk_size):
    processed_batch = process_batch(batch)
    processed_chunks.append(processed_batch)

# Concatenate all the processed chunks into a single DataFrame
result_df = pd.concat(processed_chunks)

# Print the first few rows of the result DataFrame
print(result_df.head())


       tconst     nconst category
11  tt0000005  nm0443482    actor
12  tt0000005  nm0653042    actor
16  tt0000007  nm0179163    actor
17  tt0000007  nm0183947    actor
21  tt0000008  nm0653028    actor


In [ ]:
region = pd.read_csv('title.akas.tsv.gz',sep='\t',usecols=['titleId','region'],low_memory = True,dtype={'region':'str','titleId':'str'}).set_index('titleId')['region']
region.value_counts().head(10)

DE    4400211
FR    4395613
JP    4394660
IN    4337790
ES    4315019
IT    4295836
PT    4224567
\N    1897899
US    1454840
GB     454500
Name: region, dtype: int64

In [ ]:
name = pd.read_csv('name.basics.tsv.gz', sep = '\t', low_memory=True, na_values = '\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]
name.head()

,primaryName,birthYear
nconst,,
nm0000001,Fred Astaire,1899.0
nm0000002,Lauren Bacall,1924.0
nm0000003,Brigitte Bardot,1934.0
nm0000004,John Belushi,1949.0
nm0000005,Ingmar Bergman,1918.0


In [ ]:
movies=title[title['titleType']=='movie']
cast=result_df[result_df['tconst'].isin(movies.index)]
cast.head()

,tconst,nconst,category
25,tt0000009,nm0183823,actor
26,tt0000009,nm1309758,actor
850,tt0000502,nm0215752,actor
851,tt0000502,nm0252720,actor
1044,tt0000574,nm0846894,actor


In [ ]:
def get_pairs(lang='US', min_acted=1, min_pairings=1):
    '''
    Returns an adjacency matrix and actor mapping of actor pairs where:
    - Each actor has acted in at least min_acted films
    - The two actors have acted together in at least min_pairings films
    - And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is not None:
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
        # graph = graph[graph['tconst'].isin(region[region == lang].index)]
    name_freq = graph['nconst'].value_counts()
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': square.row,
        'col': square.col,
        'n': square.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    return pairs, name.reindex(p['name'].cat.categories)

def lookup(pairs, cat):
    pairs = pd.concat([
        pairs,
        cat.iloc[pairs.row].reset_index(drop=True),
        cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)

In [ ]:
pairs, cat = get_pairs(lang='US', min_acted=1, min_pairings=1)

In [ ]:
ForKumu = lookup(pairs, cat)

In [ ]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})


In [ ]:
ForKumu.head()

,From,To,Strength
254279,Huntz Hall,Leo Gorcey,59
267090,Leo Gorcey,Huntz Hall,59
408012,Michael J. Nelson,Kevin Murphy,54
414209,Kevin Murphy,Michael J. Nelson,54
408011,Bill Corbett,Kevin Murphy,53


In [ ]:
ForKumu.shape # if the kumu website crashes when you try to upload this massive dataset, try uploading a sample of this, like the first 10,000 rows.


(1228884, 3)

In [ ]:
ForKumu_reshaped = ForKumu.head(10000)

In [ ]:
ForKumu_reshaped.to_excel("pairs.xlsx",index=False)